# Generate descriptive statistics

Also includes code for converting between measurement units.

## Setup

In [1]:
import polars as pl

In [2]:
def save(df, name):
    df.write_csv(f"../data/outputs/analyses/{name}.csv")
    return df

## Load combined records

In [3]:
records = pl.read_csv("../data/outputs/combined/records-combined.csv", infer_schema_length=None)
len(records)

823552

## Count records by sheet / year

In [4]:
(
    records
    .group_by("foia_sheet")
    .agg(
        pl.len().alias("rows"),
        pl.col("cn_qnty_yr").n_unique().alias("num_years"),
        pl.col("cn_qnty_yr").max().alias("latest_year"),
        pl.col("cn_qnty_yr").min().alias("earliest_year"),
    )
    .sort("rows", descending=True)
    .pipe(save, "row-counts-by-sheet")
)

foia_sheet,rows,num_years,latest_year,earliest_year
str,u32,u32,i64,i64
"""Pennsylvania""",95824,19,2021,2003
"""MISSING_TO_SIT…",80583,42,2020,1979
"""Missouri""",79247,26,2021,1996
"""New Hampshire""",67586,21,2020,2000
"""Illinois""",64643,43,2020,1978
…,…,…,…,…
"""Kentucky""",396,2,2020,2010
"""Alaska""",347,1,2010,2010
"""Virginia""",288,1,2010,2010


In [5]:
(
    records
    .filter(pl.col("foia_sheet").ne("MISSING_TO_SITE"))    
    .group_by("foia_sheet")
    .agg(
        pl.col("cn_qnty_yr").max().alias("latest_year"),
    )
    .group_by("latest_year")
    .agg(
        pl.len().alias("count")
    )
    .sort("count", "latest_year", descending=True)
)

latest_year,count
i64,u32
2010,16
2020,12
2021,11
2015,5
2019,2
2018,2
2014,1
2011,1


In [6]:
(
    records
    .filter(pl.col("foia_sheet").ne("MISSING_TO_SITE"))
    .group_by("cn_qnty_yr")
    .agg(
        pl.len().alias("rows"),
        pl.col("foia_sheet").n_unique().alias("num_sheets")
    )
    .sort("cn_qnty_yr", descending=True)
    .pipe(save, "row-counts-by-year")
)

cn_qnty_yr,rows,num_sheets
i64,u32,u32
2021,16304,11
2020,34063,22
2019,28857,22
2018,30350,23
2017,36062,24
…,…,…
1941,3,1
1940,3,1
1939,2,1


In [7]:
(
    records
    .filter(pl.col("cn_qnty_yr").ge(2000))
    .group_by("foia_sheet", "cn_qnty_yr")
    .agg(
        pl.len().alias("rows"),
    )
    .sort("cn_qnty_yr", descending=True)
    .pivot(
        index="foia_sheet",
        columns="cn_qnty_yr",
        values="rows",
        maintain_order=True
    )
    .fill_null(0)
    .sort("foia_sheet")
    .pipe(save, "row-counts-by-sheet-and-year")
)

foia_sheet,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""Alabama""",2,2,2,2,2,2,2,2,0,2,2,797,2,2,2,2,2,2,2,2,2,2
"""Alaska""",0,0,0,0,0,0,0,0,0,0,0,347,0,0,0,0,0,0,0,0,0,0
"""Arizona""",0,0,0,0,0,0,0,0,0,0,0,5754,0,0,0,0,0,0,0,0,0,0
"""Arkansas""",0,0,0,0,0,0,0,917,988,1072,992,1400,1048,1092,1097,1121,1090,1034,1034,946,999,975
"""California""",0,0,0,0,0,0,0,0,0,0,0,3845,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Virginia""",0,0,0,0,0,0,0,0,0,0,0,288,0,0,0,0,0,0,0,0,0,0
"""Washington""",0,0,0,0,0,0,0,0,0,0,0,1996,0,0,0,0,12,822,854,838,834,820
"""West Virginia""",8,8,8,8,8,8,7,6,10,5,4,452,0,0,0,0,0,0,0,0,0,0


## Examine redaction rates

In [8]:
redaction_proportions = (
    records
    .melt()
    .group_by("variable")
    .agg(
        (
            pl.col("value")
            .is_null()
            .mean()
            .alias("null")
        ),
        (
            pl.col("value")
            .filter(~pl.col("value").is_null())
            .str.contains(r"^\s*\(b\)")
            .mean()
            .alias("redacted")
        )
    )
    .sort("variable")
    .pipe(save, "column-nulls-and-redaction-proportions")
)

(
    redaction_proportions
    .sort("redacted", "null", "variable", descending=(True, False, False))
    .head(10)
)

variable,null,redacted
str,f64,f64
"""from_lat_va""",0.122732,0.889476
"""from_long_va""",0.122732,0.889476
"""from_dec_lat_v…",0.122836,0.889463
"""from_dec_long_…",0.122836,0.889463
"""from_site_no""",0.0,0.882239
"""from_station_n…",0.0,0.882239
"""to_site_no""",0.097848,0.842226
"""to_station_nm""",0.097848,0.842226
"""to_dec_lat_va""",0.776393,0.689479


In [9]:
(
    redaction_proportions
    .filter(pl.col("redacted").gt(0))
    .sort("redacted", descending=True)
    # .filter(pl.col("redacted").ge(0.1))
)

variable,null,redacted
str,f64,f64
"""from_lat_va""",0.122732,0.889476
"""from_long_va""",0.122732,0.889476
"""from_dec_lat_v…",0.122836,0.889463
"""from_dec_long_…",0.122836,0.889463
"""from_site_no""",0.0,0.882239
…,…,…
"""from_wu_site_c…",0.786751,0.027497
"""from_site_rmks…",0.480771,0.012755
"""to_wu_site_cm_…",0.929357,0.002492


In [10]:
(
    records
    .melt()    
    .filter(~pl.col("value").is_null())
    .filter(~pl.col("value").str.contains(r"^\s*\(b\)"))
    .group_by("variable")
    .agg(
        (
            pl.col("value")
            .n_unique()
            .alias("num_distinct")
        ),
        (
            pl.col("value")
            .first()
            .alias("example")
        ),
    )
    .sort("variable")
    .pipe(save, "column-distinct-counts")
)

variable,num_distinct,example
str,u32,str
"""accuracy_cd""",5,"""N"""
"""cn_cap_sg""",7,"""2"""
"""cn_cap_va""",961,"""60.0"""
"""cn_cd""",5,"""PI"""
"""cn_cn""",55,"""slawrenc"""
…,…,…
"""to_wu_site_mn""",56,"""slawrenc"""
"""to_zip_cd""",207,"""30901"""
"""water_cd""",8,"""SW"""


## Examine reliability codes

In [11]:
reliability_codes = (
    pl.read_excel(
        "../data/raw/dictionary/DataDictionary.xlsx",
        sheet_name="DATA RELIABILITY CODE",
        read_options={'skip_rows': 4},
    )
)
reliability_codes

Code,Name,Description
str,str,str
"""C""","""Checked""","""Data have been…"
"""L""","""Poor location""","""Location not a…"
"""M""","""Minimal""","""Minimal data."""
"""U""","""Unchecked""","""Unchecked data…"


In [12]:
(
    records
    .select("from_reliability_cd", "to_reliability_cd")
    .melt(value_name = "Code")
    .group_by("variable", "Code")
    .agg(pl.len().alias("count"))
    .sort("Code", "variable")
    .pivot(
        index="Code",
        columns="variable",
        values="count",
    )
    .fill_null(0)
    .join(reliability_codes, on = "Code")
    .pipe(save, "reliability-codes")
)

Code,from_reliability_cd,to_reliability_cd,Name,Description
str,u32,u32,str,str
"""C""",191842,41735,"""Checked""","""Data have been…"
"""L""",1350,0,"""Poor location""","""Location not a…"
"""M""",19211,5090,"""Minimal""","""Minimal data."""
"""U""",480179,554941,"""Unchecked""","""Unchecked data…"


## Count distinct measurement units used

In [13]:
(
    records
    .filter(pl.col("cn_qnty_yr").ge(2000))
    .group_by("foia_sheet", "cn_qnty_yr")
    .agg(
        pl.col("mo_unit_abbrv_tx").n_unique().alias("distinct_units"),
    )
    .filter(pl.col("distinct_units").gt(1))
    .sort("distinct_units", "cn_qnty_yr", "foia_sheet", descending=True)
)

foia_sheet,cn_qnty_yr,distinct_units
str,i64,u32
"""California""",2010,6
"""Colorado""",2010,5
"""Oklahoma""",2010,4
"""New Mexico""",2010,4
"""MISSING_TO_SIT…",2005,4
…,…,…
"""Louisiana""",2001,2
"""Georgia""",2001,2
"""Missouri""",2000,2


## Look for outliers

In [14]:
unit_conversions = pl.read_csv("../data/manual/unit-conversions.csv")
unit_conversions

unit,desc,coef_m3,coef_acre_feet
str,str,f64,f64
"""acre-inch""","""acre-inch""",102.790153,0.083333
"""acre-feet""","""acre-feet""",1233.481837,1.0
"""Tacre-feet""","""thousand acre-…",1.2335e6,1000.0
"""gal""","""gallons""",0.003785,0.000003
"""Tgal""","""thousand gallo…",3.785412,0.003069
"""Mgal""","""million gallon…",3785.411784,3.068883
"""bbl""","""barrels (42 ga…",0.158987,0.000129
"""cf""","""cubic feet""",0.028317,0.000023
"""Tcf""","""thousand cubic…",28.316847,0.022957


In [15]:
unit_to_acre_feet = dict(unit_conversions.select("unit", "coef_acre_feet").rows())
unit_to_acre_feet

{'acre-inch': 0.0833333333,
 'acre-feet': 1.0,
 'Tacre-feet': 1000.0,
 'gal': 3.0689e-06,
 'Tgal': 0.0030688833,
 'Mgal': 3.0688832772,
 'bbl': 0.0001288931,
 'cf': 2.29568e-05,
 'Tcf': 0.0229568411}

In [16]:
def with_unit_conversion(df):
    return (
        df
        .with_columns(
            pl.col("yr_unit_abbrv_tx").str.extract(r"^([^/]+)").alias("unit_volume"),
            pl.col("yr_unit_abbrv_tx").str.extract(r"/([^/]+)").alias("unit_time"),
        )
        .with_columns(
            pl.col("unit_volume").replace(unit_to_acre_feet).cast(float),
            pl.col("unit_time").replace({
                "yr": 1,
                "m": 12,
                "d": 365,
            }).cast(int),
        )
        .with_columns(
            (pl.col("unit_volume") * pl.col("unit_time")).alias("coef_acre_feet_yr")
        )
        .with_columns(
            (pl.col("cn_qnty_yr_va") * pl.col("coef_acre_feet_yr")).alias("cn_qnty_acre_feet_yr")
        )        
    )

In [17]:
(
    records
    .filter(~pl.col("cn_qnty_yr_va").is_null())
    .pipe(with_unit_conversion)
    .sort("cn_qnty_acre_feet_yr", descending=True)
    .head(10)
    .pipe(save, "largest-quantities-unitless")
)

foia_sheet,from_agency_cd,from_site_no,from_party_alias_nm,from_permit_cd,from_permit_tx,from_site_contact_nu,from_site_contact_alias_nm,from_site_contact_start_dt,from_site_contact_end_dt,from_site_contact_cn,from_site_contact_cr,from_site_contact_mn,from_site_owner_nu,from_site_owner_alias_nm,from_site_owner_cd,from_site_owner_start_dt,from_site_owner_end_dt,from_contact_party_alias_nm,from_site_owner_cn,from_site_owner_cr,from_site_owner_mn,from_site_owner_md,from_naics_cd,from_naics_ds,from_accs_cd,from_sic_cd,from_surface_water_cd,from_surface_water_nm,from_wu_site_cn,from_wu_site_cr,from_wu_site_mn,from_wu_site_md,from_wu_site_cm_tx,from_wu_site_cm_cn,from_wu_site_cm_cr,from_wu_site_cm_mn,…,cn_qnty_yr_nat_water_use_cd,cn_qnty_party_alias_nm,cn_qnty_yr,cn_qnty_pub_dt,cn_qnty_pf_fg,accuracy_cd,data_aging_cd,cn_qnty_cn,cn_qnty_cr,cn_qnty_cm_tx,cn_qnty_yr_va,cn_qnty_yr_fg,yr_unit_abbrv_tx,mo_unit_abbrv_tx,cn_qnty_yr_cn,cn_qnty_yr_cr,cn_qnty_yr_mn,cn_qnty_yr_md,cn_qnty_yr_depr_fg,cn_qnty_mo_nat_water_use_cd,cn_qnty_mo_depr_fg,cn_qnty_mo_va1,cn_qnty_mo_va2,cn_qnty_mo_va3,cn_qnty_mo_va4,cn_qnty_mo_va5,cn_qnty_mo_va6,cn_qnty_mo_va7,cn_qnty_mo_va8,cn_qnty_mo_va9,cn_qnty_mo_va10,cn_qnty_mo_va11,cn_qnty_mo_va12,unit_volume,unit_time,coef_acre_feet_yr,cn_qnty_acre_feet_yr
str,str,str,str,str,str,i64,str,i64,i64,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,…,str,str,i64,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64
"""Colorado""","""USEPA""","""(b) (7)(F)""","""USEPA""","""EPWS""","""CO0101055""",null,null,null,null,null,null,null,1,"""FEDERAL HEIGHT…","""WS""",null,null,null,"""tkrizman""","""09-OCT-15 04.4…","""tkrizman""","""09-OCT-15 04.4…",221310,"""Water Supply a…",3,null,null,null,"""ivahnenk""","""13-MAR-13 09.1…","""ivahnenk""","""13-MAR-13 09.1…",null,null,null,null,…,"""WS""","""USGS""",2010,null,"""Y""","""N""","""O""","""ldmiller""","""22-OCT-15 09.3…",null,3.25e8,"""N""","""Mgal/yr""",null,"""ldmiller""","""10/22/15 09:31…","""ldmiller""","""22-MAY-17 07.3…","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,3.068883,1,3.068883,9.9739e8
"""Georgia""","""USGS""","""31504008356000…","""GA009""","""ALLC""","""159-1112-02""",null,null,null,null,null,null,null,1,"""CRISP COUNTY P…","""GV""",null,null,null,"""slawrenc""","""24-APR-12 04.1…","""slawrenc""","""24-APR-12 04.1…",221111,"""Hydroelectric …",3,4911,"""LK""","""LAKE BLACKSHEA…","""slawrenc""","""24-APR-12 04.0…","""slawrenc""","""24-APR-12 04.0…",null,null,null,null,…,"""PH""","""GA009""",2004,null,"""Y""","""N""","""W""","""slawrenc""","""08-JAN-16 02.4…",null,22345.0,"""N""","""Mgal/d""","""Mgal/d""","""slawrenc""","""01/08/16 02:41…","""slawrenc""","""08-JAN-16 02.4…","""N""","""PH""","""N""",1842.0,3500.0,2048.0,1452.0,1232.0,1254.0,1067.0,689.0,2766.0,2270.0,2319.0,1906.0,3.068883,365,1120.142396,2.5030e7
"""South Dakota""","""USGS""","""94600000099900…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,4911,null,null,"""jnawyn""","""27-SEP-06 01.3…","""jnawyn""","""02-OCT-06 01.5…",null,null,null,null,…,"""PH""","""USCE""",2010,null,"""Y""","""G""","""W""","""lleaderc""","""19-AUG-13 02.4…","""MGD""",19352.8394,"""N""","""Mgal/d""","""Mgal/d""","""lleaderc""","""08/19/13 02:40…","""lleaderc""","""19-AUG-13 02.4…","""N""","""PH""","""N""",437.0,372.0,412.0,407.0,692.0,736.0,961.0,1152.0,1252.0,1339.0,1226.0,691.0,3.068883,365,1120.142396,2.1678e7
"""South Dakota""","""USGS""","""94680000998834…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,4911,null,null,"""jnawyn""","""27-SEP-06 01.2…","""jnawyn""","""27-SEP-06 01.2…","""MISSOURI RIVER…","""jnawyn""","""27-SEP-06 01.2…","""jnawyn""",…,"""PH""","""USCE""",2010,null,"""Y""","""G""","""W""","""lleaderc""","""19

---

---

---